In [10]:
import subprocess
import json
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

puzzle_df = pd.read_csv('lichess_db_puzzle.csv.zst', compression='zstd')

def get_concept_values_dict(fen):
    # Start the Stockfish process
    process = subprocess.Popen(
        ['stockfish-8-mac/src/stockfish_modified_evaluate'],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    # Send commands to Stockfish
    commands = f"position fen {fen}\neval\nquit\n"
    stdout, stderr = process.communicate(commands)
    
    raw_ouput = '{' + stdout[1:-4] + '}'
    json_data = json.loads(raw_ouput)

    # Capture and return the output
    return json_data

In [11]:
puzzle_df_small = puzzle_df.sample(n=100000, random_state=0)

In [12]:
%%time
concept_bool_dict = {}

for _, row in puzzle_df_small.iterrows():
    cv_dict = get_concept_values_dict(row['FEN'])
    if not cv_dict:
        continue
    
    concept_bool_dict[row['PuzzleId']] = {
        # Store FEN in dict
        'FEN': row['FEN'],
        
        # Evaluate Pieces
        'white_knight_present':                 'N' in row['FEN'],
        'white_knight_outpost':                 cv_dict['evaluate_pieces_WHITE_KNIGHT'].get('Outpost', 0) != 0,
        'white_knight_reachable_outpost':       cv_dict['evaluate_pieces_WHITE_KNIGHT'].get('ReachableOutpost', 0) != 0,
        'white_knight_minor_behind_pawn':       cv_dict['evaluate_pieces_WHITE_KNIGHT'].get('MinorBehindPawn', 0) != 0,
        'black_knight_present':                 'n' in row['FEN'],
        'black_knight_outpost':                 cv_dict['evaluate_pieces_BLACK_KNIGHT'].get('Outpost', 0) != 0,
        'black_knight_reachable_outpost':       cv_dict['evaluate_pieces_BLACK_KNIGHT'].get('ReachableOutpost', 0) != 0,
        'black_knight_minor_behind_pawn':       cv_dict['evaluate_pieces_BLACK_KNIGHT'].get('MinorBehindPawn', 0) != 0,
        
        'white_bishop_present':                 'B' in row['FEN'],
        'white_bishop_outpost':                 cv_dict['evaluate_pieces_WHITE_BISHOP'].get('Outpost', 0) != 0,
        'white_bishop_reachable_outpost':       cv_dict['evaluate_pieces_WHITE_BISHOP'].get('ReachableOutpost', 0) != 0,
        'white_bishop_minor_behind_pawn':       cv_dict['evaluate_pieces_WHITE_BISHOP'].get('MinorBehindPawn', 0) != 0,
        'black_bishop_present':                 'b' in row['FEN'],
        'black_bishop_outpost':                 cv_dict['evaluate_pieces_BLACK_BISHOP'].get('Outpost', 0) != 0,
        'black_bishop_reachable_outpost':       cv_dict['evaluate_pieces_BLACK_BISHOP'].get('ReachableOutpost', 0) != 0,
        'black_bishop_minor_behind_pawn':       cv_dict['evaluate_pieces_BLACK_BISHOP'].get('MinorBehindPawn', 0) != 0,
        
        'white_rook_present':                   'R' in row['FEN'],
        'white_rook_on_pawn':                   cv_dict['evaluate_pieces_WHITE_ROOK'].get('RookOnPawn', 0) != 0,
        'white_rook_on_file':                   cv_dict['evaluate_pieces_WHITE_ROOK'].get('RookOnFile', 0) != 0,
        'white_rook_trapped':                   cv_dict['evaluate_pieces_WHITE_ROOK'].get('TrappedRook', 0) != 0,
        'black_rook_present':                   'r' in row['FEN'],
        'black_rook_on_pawn':                   cv_dict['evaluate_pieces_BLACK_ROOK'].get('RookOnPawn', 0) != 0,
        'black_rook_on_file':                   cv_dict['evaluate_pieces_BLACK_ROOK'].get('RookOnFile', 0) != 0,
        'black_rook_trapped':                   cv_dict['evaluate_pieces_BLACK_ROOK'].get('TrappedRook', 0) != 0,
        
        'white_queen_present':                  'Q' in row['FEN'],
        'white_queen_weak':                     cv_dict['evaluate_pieces_WHITE_QUEEN'].get('WeakQueen', 0) != 0,
        'black_queen_present':                  'q' in row['FEN'],
        'black_queen_weak':                     cv_dict['evaluate_pieces_BLACK_QUEEN'].get('WeakQueen', 0) != 0,
        
        # Evaluate King
        'white_king_present':                   'K' in row['FEN'],
        'white_king_queen_check':               cv_dict['evaluate_king_WHITE'].get('QueenCheck', 0) != 0,
        'white_king_safe_check_rook':           cv_dict['evaluate_king_WHITE'].get('safe_check_ROOK', 0) != 0,
        'white_king_other_check_rook':          cv_dict['evaluate_king_WHITE'].get('other_check_ROOK', 0) != 0,
        'white_king_safe_check_bishop':         cv_dict['evaluate_king_WHITE'].get('safe_check_BISHOP', 0) != 0,
        'white_king_other_check_bishop':        cv_dict['evaluate_king_WHITE'].get('other_check_BISHOP', 0) != 0,
        'white_king_safe_check_knight':         cv_dict['evaluate_king_WHITE'].get('safe_check_KNIGHT', 0) != 0,
        'white_king_other_check_knight':        cv_dict['evaluate_king_WHITE'].get('other_check_KNIGHT', 0) != 0,
        'white_king_pawnless_flank':            cv_dict['evaluate_king_WHITE'].get('PawnlessFlank', 0) != 0,
        'black_king_present':                   'k' in row['FEN'],
        'black_king_queen_check':               cv_dict['evaluate_king_BLACK'].get('QueenCheck', 0) != 0,
        'black_king_safe_check_rook':           cv_dict['evaluate_king_BLACK'].get('safe_check_ROOK', 0) != 0,
        'black_king_other_check_rook':          cv_dict['evaluate_king_BLACK'].get('other_check_ROOK', 0) != 0,
        'black_king_safe_check_bishop':         cv_dict['evaluate_king_BLACK'].get('safe_check_BISHOP', 0) != 0,
        'black_king_other_check_bishop':        cv_dict['evaluate_king_BLACK'].get('other_check_BISHOP', 0) != 0,
        'black_king_safe_check_knight':         cv_dict['evaluate_king_BLACK'].get('safe_check_KNIGHT', 0) != 0,
        'black_king_other_check_knight':        cv_dict['evaluate_king_BLACK'].get('other_check_KNIGHT', 0) != 0,
        'black_king_pawnless_flank':            cv_dict['evaluate_king_BLACK'].get('PawnlessFlank', 0) != 0,
        
        # Evaluate Threats
        'white_threats_loose_enemies':          cv_dict['evaluate_threats_WHITE'].get('LooseEnemies', 0) != 0,
        'white_threats_weak_one_bits':          cv_dict['evaluate_threats_WHITE'].get('weak_1_bits', 0) != 0,
        'white_threats_threat_by_hanging_pawn': cv_dict['evaluate_threats_WHITE'].get('ThreatByHangingPawn', 0) != 0,
        'white_threats_threat_by_safe_pawn':    cv_dict['evaluate_threats_WHITE'].get('ThreatBySafePawn', 0) != 0,
        'white_threats_defended_bits':          cv_dict['evaluate_threats_WHITE'].get('defended_bits', 0) != 0,
        'white_threats_weak_two_bits':          cv_dict['evaluate_threats_WHITE'].get('weak_2_bits', 0) != 0,
        'white_threats_threat_by_pawn_push':    cv_dict['evaluate_threats_WHITE'].get('ThreatByPawnPush', 0) != 0,
        
        'black_threats_loose_enemies':          cv_dict['evaluate_threats_BLACK'].get('LooseEnemies', 0) != 0,
        'black_threats_weak_one_bits':          cv_dict['evaluate_threats_BLACK'].get('weak_1_bits', 0) != 0,
        'black_threats_threat_by_hanging_pawn': cv_dict['evaluate_threats_BLACK'].get('ThreatByHangingPawn', 0) != 0,
        'black_threats_threat_by_safe_pawn':    cv_dict['evaluate_threats_BLACK'].get('ThreatBySafePawn', 0) != 0,
        'black_threats_defended_bits':          cv_dict['evaluate_threats_BLACK'].get('defended_bits', 0) != 0,
        'black_threats_weak_two_bits':          cv_dict['evaluate_threats_BLACK'].get('weak_2_bits', 0) != 0,
        'black_threats_threat_by_pawn_push':    cv_dict['evaluate_threats_BLACK'].get('ThreatByPawnPush', 0) != 0,
        
        # Other Functions
        'white_passed_pawns':                   cv_dict['evaluate_passed_pawns_WHITE'].get('score', 0) != 0,
        'black_passed_pawns':                   cv_dict['evaluate_passed_pawns_BLACK'].get('score', 0) != 0,
    }

CPU times: user 43.7 s, sys: 23min 28s, total: 24min 12s
Wall time: 14h 52min 9s


In [13]:
concept_bool_df = pd.DataFrame.from_dict(concept_bool_dict, orient='index')
concept_bool_df.iloc[:, 1:] = concept_bool_df.iloc[:, 1:].apply(lambda col: col.astype(int))
display(concept_bool_df.describe().T.describe())

,count,mean,std,min,25%,50%,75%,max
count,62.0,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.0
mean,99986.0,0.251002,0.283589,0.032258,0.112903,0.209677,0.306452,1.0
std,0.0,0.295264,0.148866,0.178127,0.319058,0.410402,0.464783,0.0
min,99986.0,0.000170,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,99986.0,0.033965,0.165818,0.000000,0.000000,0.000000,0.000000,1.0
50%,99986.0,0.106275,0.295752,0.000000,0.000000,0.000000,0.000000,1.0
75%,99986.0,0.373367,0.407065,0.000000,0.000000,0.000000,1.000000,1.0
max,99986.0,1.000000,0.499765,1.000000,1.000000,1.000000,1.000000,1.0


In [14]:
concept_bool_df.to_csv('stockfish_boolean_concepts_100000.csv')

### Descripton of the boolean concepts present here

In [6]:
'<color>_<piece>_present' : 
    'FEN determination of whether or not piece is present'

'<color>_<knight/bishop>_outpost' : 
    '''
    Outpost[knight/bishop][supported by pawn] contains bonuses for knights and
    bishops outposts, bigger if outpost piece is supported by a pawn.
    '''

'<color>_<knight/bishop>_reachable_outpost' :

    '''
    ReachableOutpost[knight/bishop][supported by pawn] contains bonuses for
    knights and bishops which can reach an outpost square in one move, bigger
    if outpost square is supported by a pawn.
    '''

'<color>_<knight/bishop>_minor_behind_pawn' : 
    '''
    Bonus when behind a pawn
    '''
    
'<color>_rook_on_pawn': 
    '''
    Bonus for aligning with enemy pawns on the same rank/file
    '''

'<color>_rook_on_file': 
    '''
    Bonus when on an open or semi-open file
    '''

'<color>_rook_trapped': 
    '''
    Penalize when trapped by the king, even more if the king cannot castle
    '''
    
'<color>_queen_weak': 
    '''
    Penalty if any relative pin or discovered attack against the queen
    '''

'<color>_king_queen_check':
    '''
    Enemy queen safe checks
    '''
    
'<color>_king_safe_check_<rook/bishop/knight>':
    '''
    Enemy piece safe checks
    '''
    
'<color>_king_other_check_<rook/bishop/knight>':
    '''
    Enemy piece other checks
    '''
    
'<color>_king_pawnless_flank':
    '''
    Penalty when our king is on a pawnless flank
    '''
  
'<color>_threats_loose_enemies':
    '''
    Small bonus if the opponent has loose pawns or pieces
    '''

'<color>_threats_weak_one_bits':
    '''
    Presence of non-pawn enemies attacked by a pawn
    '''

'<color>_threats_threat_by_hanging_pawn':
    '''
    NO SPECIFICATION
    
    NOTE: Relates to weak_one_bits
    '''

'<color>_threats_threat_by_safe_pawn':
    '''
    ThreatBySafePawn[PieceType] contains bonuses according to which piece
    type is attacked by a pawn which is protected or is not attacked.
    
    NOTE: Relates to weak_one_bits
    '''

'<color>_threats_defended_bits':
    '''
    Non-pawn enemies defended by a pawn
    '''

'<color>_threats_weak_two_bits':
    '''
    Presence of enemies not defended by a pawn and under our attack
    '''
    
'<color>_threats_threat_by_pawn_push':
    '''
    Bonus if some pawns can safely push and attack an enemy piece
    '''

'<color>_passed_pawns':
    '''
    Evaluation of the passed pawns of the given color
    '''

SyntaxError: invalid syntax (3558613814.py, line 1)

In [87]:
puzzle_df_small.iloc[626, 1]
# get_concept_values_dict(puzzle_df_small.iloc[626, 1])

'6Q1/7K/2k5/3r4/8/8/8/8 w - - 5 82'

### old

In [ ]:
import json

fen = "rn2kb1r/pp3p1p/2b1PB2/2p5/3P4/5N2/q4PPP/R2QKBNR b KQkq - 0 1"
concept_values_dict = get_concept_values_dict(fen)

In [ ]:
import subprocess

def get_concept_values(fen):
    # Start the Stockfish process
    process = subprocess.Popen(
        ['stockfish-8-mac/src/stockfish'],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    # Send commands to Stockfish
    commands = f"position fen {fen}\neval\nquit\n"
    stdout, stderr = process.communicate(commands)

    # Capture and return the output
    return stdout.strip()

# Example FEN
# fen = "rnbqkb1r/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1"
fen = "rn2kb1r/pp3p1p/2b1PB2/2p5/3P4/5N2/q4PPP/R2QKBNR b KQkq - 0 1"
fen = "rn2kb1r/pp3p1p/2b1PB2/q1p5/3P4/5N2/5PPP/R2QKBNR w KQkq - 0 1"
stdout = get_concept_values(fen)

print("Standard Output:\n", stdout)
print("Standard Error:\n", stderr)


In [ ]:
%%time

def process_row(row):
    cv_dict = get_concept_values_dict(row['FEN'])
    if not cv_dict:
        return None
    
    return {
        row['PuzzleId']: {
            # Store FEN in dict
            'FEN': row['FEN'],

            # Evaluate Pieces
            'white_knight_present':                 'N' in row['FEN'],
            'white_knight_outpost':                 cv_dict['evaluate_pieces_WHITE_KNIGHT'].get('Outpost', 0) != 0,
            'white_knight_reachable_outpost':       cv_dict['evaluate_pieces_WHITE_KNIGHT'].get('ReachableOutpost', 0) != 0,
            'white_knight_minor_behind_pawn':       cv_dict['evaluate_pieces_WHITE_KNIGHT'].get('MinorBehindPawn', 0) != 0,
            'black_knight_present':                 'n' in row['FEN'],
            'black_knight_outpost':                 cv_dict['evaluate_pieces_BLACK_KNIGHT'].get('Outpost', 0) != 0,
            'black_knight_reachable_outpost':       cv_dict['evaluate_pieces_BLACK_KNIGHT'].get('ReachableOutpost', 0) != 0,
            'black_knight_minor_behind_pawn':       cv_dict['evaluate_pieces_BLACK_KNIGHT'].get('MinorBehindPawn', 0) != 0,

            'white_bishop_present':                 'B' in row['FEN'],
            'white_bishop_outpost':                 cv_dict['evaluate_pieces_WHITE_BISHOP'].get('Outpost', 0) != 0,
            'white_bishop_reachable_outpost':       cv_dict['evaluate_pieces_WHITE_BISHOP'].get('ReachableOutpost', 0) != 0,
            'white_bishop_minor_behind_pawn':       cv_dict['evaluate_pieces_WHITE_BISHOP'].get('MinorBehindPawn', 0) != 0,
            'black_bishop_present':                 'b' in row['FEN'],
            'black_bishop_outpost':                 cv_dict['evaluate_pieces_BLACK_BISHOP'].get('Outpost', 0) != 0,
            'black_bishop_reachable_outpost':       cv_dict['evaluate_pieces_BLACK_BISHOP'].get('ReachableOutpost', 0) != 0,
            'black_bishop_minor_behind_pawn':       cv_dict['evaluate_pieces_BLACK_BISHOP'].get('MinorBehindPawn', 0) != 0,

            'white_rook_present':                   'R' in row['FEN'],
            'white_rook_on_pawn':                   cv_dict['evaluate_pieces_WHITE_ROOK'].get('RookOnPawn', 0) != 0,
            'white_rook_on_file':                   cv_dict['evaluate_pieces_WHITE_ROOK'].get('RookOnFile', 0) != 0,
            'white_rook_trapped':                   cv_dict['evaluate_pieces_WHITE_ROOK'].get('TrappedRook', 0) != 0,
            'black_rook_present':                   'r' in row['FEN'],
            'black_rook_on_pawn':                   cv_dict['evaluate_pieces_BLACK_ROOK'].get('RookOnPawn', 0) != 0,
            'black_rook_on_file':                   cv_dict['evaluate_pieces_BLACK_ROOK'].get('RookOnFile', 0) != 0,
            'black_rook_trapped':                   cv_dict['evaluate_pieces_BLACK_ROOK'].get('TrappedRook', 0) != 0,

            'white_queen_present':                  'Q' in row['FEN'],
            'white_queen_weak':                     cv_dict['evaluate_pieces_WHITE_QUEEN'].get('WeakQueen', 0) != 0,
            'black_queen_present':                  'q' in row['FEN'],
            'black_queen_weak':                     cv_dict['evaluate_pieces_BLACK_QUEEN'].get('WeakQueen', 0) != 0,

            # Evaluate King
            'white_king_present':                   'K' in row['FEN'],
            'white_king_queen_check':               cv_dict['evaluate_king_WHITE'].get('QueenCheck', 0) != 0,
            'white_king_safe_check_rook':           cv_dict['evaluate_king_WHITE'].get('safe_check_ROOK', 0) != 0,
            'white_king_other_check_rook':          cv_dict['evaluate_king_WHITE'].get('other_check_ROOK', 0) != 0,
            'white_king_pawnless_flank':            cv_dict['evaluate_king_WHITE'].get('PawnlessFlank', 0) != 0,
            'black_king_present':                   'k' in row['FEN'],
            'black_king_queen_check':               cv_dict['evaluate_king_BLACK'].get('QueenCheck', 0) != 0,
            'black_king_safe_check_rook':           cv_dict['evaluate_king_BLACK'].get('safe_check_ROOK', 0) != 0,
            'black_king_other_check_rook':          cv_dict['evaluate_king_BLACK'].get('other_check_ROOK', 0) != 0,
            'black_king_pawnless_flank':            cv_dict['evaluate_king_BLACK'].get('PawnlessFlank', 0) != 0,

            # Evaluate Threats
            'white_threats_loose_enemies':          cv_dict['evaluate_threats_WHITE'].get('LooseEnemies', 0) != 0,
            'white_threats_weak_one_bits':          cv_dict['evaluate_threats_WHITE'].get('weak_1_bits', 0) != 0,
            'white_threats_threat_by_hanging_pawn': cv_dict['evaluate_threats_WHITE'].get('ThreatByHangingPawn', 0) != 0,
            'white_threats_threat_by_safe_pawn':    cv_dict['evaluate_threats_WHITE'].get('ThreatBySafePawn', 0) != 0,
            'white_threats_defended_bits':          cv_dict['evaluate_threats_WHITE'].get('defended_bits', 0) != 0,
            'white_threats_weak_two_bits':          cv_dict['evaluate_threats_WHITE'].get('weak_2_bits', 0) != 0,
            'white_threats_threat_by_pawn_push':    cv_dict['evaluate_threats_WHITE'].get('ThreatByPawnPush', 0) != 0,

            'black_threats_loose_enemies':          cv_dict['evaluate_threats_BLACK'].get('LooseEnemies', 0) != 0,
            'black_threats_weak_one_bits':          cv_dict['evaluate_threats_BLACK'].get('weak_1_bits', 0) != 0,
            'black_threats_threat_by_hanging_pawn': cv_dict['evaluate_threats_BLACK'].get('ThreatByHangingPawn', 0) != 0,
            'black_threats_threat_by_safe_pawn':    cv_dict['evaluate_threats_BLACK'].get('ThreatBySafePawn', 0) != 0,
            'black_threats_defended_bits':          cv_dict['evaluate_threats_BLACK'].get('defended_bits', 0) != 0,
            'black_threats_weak_two_bits':          cv_dict['evaluate_threats_BLACK'].get('weak_2_bits', 0) != 0,
            'black_threats_threat_by_pawn_push':    cv_dict['evaluate_threats_BLACK'].get('ThreatByPawnPush', 0) != 0,

            # Other Functions
            'white_passed_pawns':                   cv_dict['evaluate_passed_pawns_WHITE'].get('score', 0) != 0,
            'black_passed_pawns':                   cv_dict['evaluate_passed_pawns_BLACK'].get('score', 0) != 0,
        }
    }

if __name__ == '__main__':
    concept_bool_dict = {}
    with ProcessPoolExecutor() as executor:
        results = executor.map(process_row, [row for _, row in puzzle_df_small.iterrows()])

    for result in results:
        if result is not None:
            concept_bool_dict.update(result)